In [6]:
import random
import pandas as pd
import os
import math
import scipy
import numpy as np

In [7]:
class HholdAgent:
    """An agent with fixed initial money/food etc."""
    def __init__(self, unique_id, loc, supermarket, online_shop):


        # fixed characteristics
        self.hholdsize = 1
        self.loc = loc
        self.unique_id = unique_id
        self.supermarket_all = supermarket
        self.demand_mult = 1
        
        # identify whether work in city centre or not
        self.loc_work_centre = loc_info.loc[loc_info['name'] == self.loc, 'work_centre'].values[0]
        if random.random() <= self.loc_work_centre:
            self.work_centre = 1
        else: 
            self.work_centre = 0


        # Only available online shop is the one for their area
        self.online_shop = online_shop[self.loc-1]

        # variable
        self.cupboard = random.randint(0,(100*self.demand_mult))
        self.failed_shops = 0
        self.amountshop = 0
        
        # var showing online shopping preference - initialised with location data
        self.online = loc_info.loc[loc_info['name'] == self.loc, 'perc_online'].values[0]
        
        
        # initialise individual supermarket preferences
        self.superm_simil = []
        
        for j in range(len(supermarket)):
            if shop_info.loc[shop_info['type'] == "supermarket", 'location'].iloc[j] == self.loc:
                self.superm_simil.append(2.5)
                
            elif (shop_info.loc[shop_info['type'] == "supermarket", 'location'].iloc[j] == 1 and \
                 self.work_centre == 1):
                self.superm_simil.append(2.5)
                
            else:
                self.superm_simil.append(0)
        

            
    
        
        
        
        # create history lists
#         self.history_eat = []
        self.history_cupboard = []
        self.history_amountshop = []
        
#         self.history_whereshop = []
        self.history_success = []
        self.history_purchased = []
        
        self.history_online_pref = []

#         self.descript = [self.unique_id, self.loc, self.hholdsize]

        
        

        # once I add in multiple shops, then have utility for each (and logit fn?)
        #self.shop_memory = []

        
        
# for initial model just binary shop or not,
    # but later extend to have non-deterministic prob of shop and varying shop requirements
    def howmuch_shop(self):
        
        self.history_cupboard.append(self.cupboard)
        self.history_online_pref.append(self.online)
       
        if self.cupboard < ((0.4*100*self.demand_mult)*random.uniform(0.5,1.5)):
            
            self.amountshop = math.floor(((100*self.demand_mult) - self.cupboard) + (5*self.failed_shops))
            
            self.history_amountshop.append(self.amountshop)
            return self.amountshop
        else:
            
            self.history_amountshop.append(0)  
#             self.history_whereshop.append("") 
            self.history_success.append("")             
            self.history_purchased.append(0)                
                
            return 0
    

#  use shop.memory to introduce preferences and location choices
    def where_shop(self):
        if self.online > random.random():
#             self.history_whereshop.append("O")
            return 'online'
        else:
#             self.history_whereshop.append("S")
            
            
            # select which store to use:
            
            # recalc probs
            self.superm_pref = []
        
            for i in range(len(self.supermarket_all)):
                temp_prob_superm = (math.exp(self.superm_simil[i])/(sum([math.exp(simil1) for simil1 in self.superm_simil])))
                
                self.superm_pref.append(temp_prob_superm)
                
            #self.superm_pref = scipy.special.softmax()
            
            if random.random() < 0.95:
                self.supermarket = self.supermarket_all[random.choices(range(len(self.supermarket_all)), weights = self.superm_pref, k=1)[0]]
            else:
                self.supermarket = self.supermarket_all[random.choice(range(len(self.supermarket_all)))]
 
            
            return 'supermarket'



    def eat(self):
                
        stock_change =  random.randint(min(math.floor(10*self.demand_mult),self.cupboard), \
                                           min(math.floor(20*self.demand_mult),self.cupboard))
        self.cupboard -= stock_change
        
#         self.history_eat.append(stock_change)


        

    def run_day(self):
        ''' Do todays shopping then decide what shop to do tomorrow'''


        # do shopping
        if self.howmuch_shop() > 0:
            where = self.where_shop()

            if where == 'supermarket':
                self.supermarket.accept(self)
            if where == 'online':
                self.online_shop.accept(self)

        self.eat()
        

        
    def start_pandemic_hh(self, demand_mult2, wfh2):
        
        self.demand_mult = demand_mult2
        
        if (self.work_centre == 1 and self.loc != 1):
            
            if random.random() > wfh2:
                for j in range(len(self.supermarket_all)):
                    if shop_info.loc[shop_info['type'] == "supermarket", 'location'].iloc[j] == 1:
                        self.superm_simil[j] = 0


 
        
        
        
        
        


In [8]:
class Shop():

    def __init__(self, unique_id, type, daily_stock, capacity):

        self.daily_stock = daily_stock
        self.capacity = capacity
        self.type = type
        self.unique_id = unique_id
        self.queue = []
        
        self.history_end_day_stock = []
        


    def accept(self, household):
        self.queue.append(household)

    def serves_everyone(self):
        self.dynamic_stock = self.daily_stock
        
        # some people immediately drop out as too busy?
        
        

        # need to reorder as hholds always processed in same order
        random.shuffle(self.queue)
        
 
        # Check who can be served
    
        if len(self.queue) <= self.capacity:
            enter_store = self.queue
        else:
            enter_store = self.queue[:self.capacity]
        
        
        # for those allocated a place, need to check stock levels:
        
        for hh in enter_store:
            
                # if store has no stock left
                if self.dynamic_stock == 0:
                    hh.failed_shops += 1
                    
                    hh.history_success.append("UnSuc_" + str(self.type) + "_" + str(self.unique_id))             
                    
                    hh.history_purchased.append(0) 
                    
                    # update preferences
                    if self.type == "online":
                        hh.online = max(hh.online - 0.1, 0.05)
                    elif self.type == "supermarket":
                        hh.superm_simil[self.unique_id] = max(hh.superm_simil[self.unique_id] - 0.5, 0)   

                # if store can fully satisfy needs of the shopper
                elif hh.amountshop <= self.dynamic_stock:
                    hh.cupboard += hh.amountshop
                    self.dynamic_stock -= hh.amountshop

                    hh.history_success.append("Suc_" + str(self.type) + "_" + str(self.unique_id))             
                    hh.history_purchased.append(hh.amountshop) 
                    
                    hh.failed_shops = 0
                    
                    if self.type == "online":
                        hh.online = min(hh.online + 0.02, 0.5)
                    elif self.type == "supermarket":
                        hh.superm_simil[self.unique_id] = min(hh.superm_simil[self.unique_id] + 0.5, 5)    

                # if store can partially satisfy needs of the shopper                  
                elif hh.amountshop > self.dynamic_stock:
                    hh.cupboard += self.dynamic_stock
  
                    hh.history_success.append("Part_" + str(self.type) + "_" + str(self.unique_id))             
                    hh.history_purchased.append(self.dynamic_stock)
                    
                    hh.failed_shops = 0
                    
                    self.dynamic_stock = 0
                    

          # hholds not allocated a spot
        if len(self.queue) > self.capacity:
            for hh in self.queue[self.capacity:]:

                hh.failed_shops += 1

                hh.history_success.append("UnSuc_" + str(self.type) + "_" + str(self.unique_id))             
                hh.history_purchased.append(0) 
                
                # update customer's store preferences
                if self.type == "online":
                    hh.online = max(hh.online - 0.1, 0.05)
                elif self.type == "supermarket":
                    hh.superm_simil[self.unique_id] = max(hh.superm_simil[self.unique_id] - 0.5, 0) 

        self.queue = []
        
#        self.end_of_day = (str(self.type) + "_" + str(self.unique_id), self.dynamic_stock)
#        self.history_end_day_stock.append(self.end_of_day)
        
        self.history_end_day_stock.append(self.dynamic_stock)
        
        
    def start_pandemic_store(self, cap_changeO, cap_changeS):
        
        if self.type == "online":
            self.capacity += math.ceil(self.capacity*cap_changeO)
        
        elif self.type == "supermarket":
            self.capacity += math.floor(self.capacity*cap_changeS)




 



In [9]:
# to run the model itself
def run(run_num, demand_mult2, wfh2, cap_changeO, cap_changeS):
    
    # set up output files
#     all_history_eat = []
    all_history_cupboard = []
    all_history_amountshop = []
        
#     all_history_whereshop = []
    all_history_success = []
    all_history_purchased = []
    
    all_history_online_pref = []

    all_history_shop_end_day_stock = []

#     all_history_descript = []
    
    
    
    
    # creating shops
    #all_supermarkets = []
    #all_online = []
            
    all_supermarkets = [Shop(i, \
                             'supermarket', \
                             shop_info.loc[shop_info['type'] == "supermarket",'stock'].iloc[i], \
                             shop_info.loc[shop_info['type'] == "supermarket",'capacity'].iloc[i]) \
                        for i in range(len(shop_info.loc[shop_info['type'] == "supermarket"].index))]
        
    all_online = [Shop(i, \
                             'online', \
                             shop_info.loc[shop_info['type'] == "online",'stock'].iloc[i], \
                             shop_info.loc[shop_info['type'] == "online",'capacity'].iloc[i]) \

                        for i in range(len(shop_info.loc[shop_info['type'] == "online"].index))]

     
    #print(all_online[1])
    
    
    num_online = len(all_online)
    num_super = len(all_supermarkets)
    

    
    # creating household agents

    all_locs = []
    
    for num_locs in range(len(loc_info.index)):
        all_locs.extend([loc_info.loc[num_locs,'name']]*loc_info.loc[num_locs,'hholds'])
          
    #all_households = []
    all_households = [HholdAgent(i,  \
                                 all_locs[i], \
                                 all_supermarkets, all_online) for i in range(loc_info['hholds'].sum())]





    
    # run shopping fn
    
    # for baseline (business as usual) period
    for day in range(1,76):
                
        for hh in all_households:
            hh.run_day()
            
        # no need to reorder as each place indep - only one shop attempt per day
        for on_sup in range(num_online):
            all_online[on_sup].serves_everyone()
            
        for in_sup in range(num_super):
            all_supermarkets[in_sup].serves_everyone()

            
            
            
        
    # to shift to disruption state:
    for hh in all_households:
        hh.start_pandemic_hh(demand_mult2, wfh2)

    for on_sup in all_online:
        on_sup.start_pandemic_store(cap_changeO, cap_changeS)

    for in_sup in all_supermarkets:
        in_sup.start_pandemic_store(cap_changeO, cap_changeS)
           


            
            
            
            
    # for days during disruption      
    for day in range(76,num_days+ 1):
        
        for hh in all_households:
            hh.run_day()
            
        # shop during disruption
        for on_sup in range(num_online):
            all_online[on_sup].serves_everyone()
            
        for in_sup in range(num_super):
            all_supermarkets[in_sup].serves_everyone()
            
            
            
            
            
            

    # Combine outputs for all households
    for hh in all_households:
#         all_history_eat.append(hh.history_eat)  
        all_history_cupboard.append(hh.history_cupboard)  
        all_history_amountshop.append(hh.history_amountshop)  
#         all_history_whereshop.append(hh.history_whereshop)  
        all_history_success.append(hh.history_success)  
        all_history_purchased.append(hh.history_purchased) 
        all_history_online_pref.append(hh.history_online_pref) 
#         all_history_descript.append(hh.descript) 

        
    for on_sup in all_online:
        on_sup.EOD = (str(on_sup.type) + "_" + str(on_sup.unique_id), on_sup.history_end_day_stock)
        all_history_shop_end_day_stock.append(on_sup.EOD) 

    for in_sup in all_supermarkets:
        in_sup.EOD = (str(in_sup.type) + "_" + str(in_sup.unique_id), in_sup.history_end_day_stock)
        all_history_shop_end_day_stock.append(in_sup.EOD) 
       

    
    # add details of sim to summary df
    overall_sim = pd.DataFrame({'name' : [sim_name],
                                'run' : [run_num],
                                'num_locs' : [len(loc_info.index)],
                                'num_days' : [num_days],
                                'seed' : [seed1],
                                'num_agents' : loc_info['hholds'].sum(),
                                'super_stock' : shop_info.loc[shop_info['type'] == "supermarket", 'stock'].sum(),
                                'super_capacity' : shop_info.loc[shop_info['type'] == "supermarket", 'capacity'].sum(),
                                'online_stock' : shop_info.loc[shop_info['type'] == "online", 'stock'].sum(),
                                'online_capacity' : shop_info.loc[shop_info['type'] == "online", 'capacity'].sum(),
                                'Disruption - demand multiplier' : [demand_mult2],
                                'Disruption - % workers still going out to work' : [wfh2],
                                'Disruption - in-store cap change' : [cap_changeS],
                                'Disruption - online cap change' : [cap_changeO]})

    if k == 1:
        overall_sim.to_csv('sim_overview2.csv', mode='a', header=not os.path.exists('sim_overview2.csv'), index = False) 
    
    
    # then write the other history files
    # store all in one folder
    os.makedirs(sim_name, exist_ok=True)  
      
#    pd_all_hist_eat = pd.DataFrame(all_history_eat, columns = ["day_" + str(i) for i in range(1,num_days + 1)]) 
    pd_all_hist_cupboard = pd.DataFrame(all_history_cupboard, columns = ["day_" + str(i) for i in range(1,num_days + 1)]) 
    pd_all_hist_amountshop = pd.DataFrame(all_history_amountshop, columns = ["day_" + str(i) for i in range(1,num_days + 1)])  
#    pd_all_hist_whereshop = pd.DataFrame(all_history_whereshop, columns = ["day_" + str(i) for i in range(1,num_days + 1)])  
    pd_all_hist_success = pd.DataFrame(all_history_success, columns = ["day_" + str(i) for i in range(1,num_days + 1)]) 
    pd_all_hist_purchased = pd.DataFrame(all_history_purchased, columns = ["day_" + str(i) for i in range(1,num_days + 1)])  
    pd_all_hist_online_pref = pd.DataFrame(all_history_online_pref, columns = ["day_" + str(i) for i in range(1,num_days + 1)])  
#    pd_all_hist_descript = pd.DataFrame(all_history_descript, columns = ["ID", "location", "hhold_size"])  
   
    pd_all_history_shop_end_day_stock = pd.DataFrame([sub[1] for sub in all_history_shop_end_day_stock], \
                                                     columns = ["day_" + str(i) for i in range(1,num_days + 1)])


    
    # do relevant calcs and formatting
    pd_all_hist_amountshop2 = pd_all_hist_amountshop.apply(sum, result_type='expand').to_frame().T
    pd_all_hist_purchased2 = pd_all_hist_purchased.apply(sum, result_type='expand').to_frame().T
    pd_all_hist_cupboardMEAN = pd_all_hist_cupboard.mean(axis=0, skipna = True, numeric_only = True).to_frame().T
    pd_all_hist_cupboardVAR = pd_all_hist_cupboard.var(axis=0, skipna = True, numeric_only = True).to_frame().T
    pd_all_hist_cupboardEMPTY = pd_all_hist_cupboard[pd_all_hist_cupboard == 0].count().to_frame().T
    pd_all_hist_success2 = pd_all_hist_success.aggregate(pd.value_counts)
        
    # add in col so I know the run number
    pd_all_hist_amountshop2.insert(0, 'run_num', [run_num])
    pd_all_hist_purchased2.insert(0, 'run_num', [run_num])
    pd_all_hist_cupboardMEAN.insert(0, 'run_num', [run_num])
    pd_all_hist_cupboardVAR.insert(0, 'run_num', [run_num])
    pd_all_hist_cupboardEMPTY.insert(0, 'run_num', [run_num])
    pd_all_hist_success2.insert(0, 'run_num', [run_num]*len(pd_all_hist_success.aggregate(pd.value_counts).index))

    pd_all_history_shop_end_day_stock.insert(0, 'shop', [sub[0] for sub in all_history_shop_end_day_stock])
    pd_all_history_shop_end_day_stock.insert(1, 'run_num', [run_num]*len(shop_info))


    # write all to csv
    if run_num == 1:
        pd_all_hist_amountshop2.to_csv(sim_name + '/hist_totshop.csv', mode = 'w', header = True, index = False) 
        pd_all_hist_purchased2.to_csv(sim_name + '/hist_totpurch.csv', mode = 'w', header = True, index = False) 
        pd_all_hist_cupboardMEAN.to_csv(sim_name + '/hist_avcupboard.csv', mode = 'w', header = True, index = False) 
        pd_all_hist_cupboardVAR.to_csv(sim_name + '/hist_varcupboard.csv', mode = 'w', header = True, index = False) 
        pd_all_hist_cupboardEMPTY.to_csv(sim_name + '/hist_emptycupboard.csv', mode = 'w', header = True, index = False) 
        pd_all_hist_success2.to_csv(sim_name + '/hist_successes.csv', mode = 'w', header = True) 
        
        pd_all_history_shop_end_day_stock.to_csv(sim_name + '/hist_EndOfDay.csv', mode = 'w', header = True) 

    else:
        pd_all_hist_amountshop2.to_csv(sim_name + '/hist_totshop.csv', mode = 'a', header = False, index = False) 
        pd_all_hist_purchased2.to_csv(sim_name + '/hist_totpurch.csv', mode = 'a', header = False, index = False) 
        pd_all_hist_cupboardMEAN.to_csv(sim_name + '/hist_avcupboard.csv', mode = 'a', header = False, index = False) 
        pd_all_hist_cupboardVAR.to_csv(sim_name + '/hist_varcupboard.csv', mode = 'a', header = False, index = False) 
        pd_all_hist_cupboardEMPTY.to_csv(sim_name + '/hist_emptycupboard.csv', mode = 'a', header = False, index = False) 
        pd_all_hist_success2.to_csv(sim_name + '/hist_successes.csv', mode = 'a', header = False) 
        pd_all_history_shop_end_day_stock.to_csv(sim_name + '/hist_EndOfDay.csv', mode = 'a', header = False) 


    
#     pd_all_hist_success.to_csv(sim_name + '/hist_indiv_successes.csv', mode = 'w', header = True, index = False) 
#     pd_all_hist_online_pref.to_csv(sim_name + '/hist_onlinepref.csv', mode = 'w', header = True, index = False) 

    


In [11]:
##############################################################################
## global params:

sim_nameALL = "Final12_socialdist_75PERCred_"
num_days = 365



# read in input files:
loc_info = pd.read_csv('ABM_input_locs4.csv')
shop_info = pd.read_csv('ABM_input_shops4.csv')


# pandemic settings:
demand_mult2 = 1          # 1 for no change
wfh2 = 1                  # 1 for no change
cap_changeO = 0           # 0 for no change
cap_changeS = -0.75           # 0 for no change

#n = 1

for n in range(1,11):
    seed1 = 6482*n

    random.seed(seed1)
    
    sim_name = sim_nameALL + "_" + str(n)

    for k in range(1, 101):                 # do 1 to 1001
        run(k, demand_mult2, wfh2, cap_changeO, cap_changeS)

        if (k % 50) == 0:
            print(k)

50
100
50
100
50
100
50
100
50
100
50
100
50
100
50
100
50
100
50
100


In [16]:
print(pd_all_hist_cupboardMEAN)

NameError: name 'pd_all_hist_cupboardMEAN' is not defined

In [6]:
shop_info = pd.read_csv('ABM_input_shops4.csv')

print(len(shop_info))

8


In [9]:
n = 1
seed1 = 6482*n

print(range(100*(n-1), 100*n))

range(0, 100)


   run_num  a  b  c
0        1  6  6  6
